In [51]:
import pandas as pd
import json
import re
from urllib.parse import unquote
import discogs_client as DiscogsClient
DISCOGS_USER_TOKEN='TFVOBtHcTkDzNyzLtYgqdJteXoXSlKyCjVtnjDNW'
SPOTIFY_CLIENT_ID='6f4d5e1238de4803bca434bb32da9ad5'
SPOTIFY_CLIENT_SECRET='37c32729befa4188b715d6c30135c9e4'
YTMUSIC_ACCESS_TOKEN='ya29.a0AfB_byB9CWG5iwkYBMZeaHiGHguWqsXF2vy3aYdugq1dPKsFT05fWp54cp97fuKATy_fQbe7qoPIxSSMsKxVAM8fbWzdteGNY-8m_J7gMC_df-H6YSYVCiMktQbpQv-7qH53f0zfxdZzm4g8vMElIJ9h5ly4t0MegtuUtKxl1znLkCEqaCgYKAdYSARISFQHGX2Mi-A93ch6xphVdmKGeueBcLw0183'
YTMUSIC_REFRESH_TOKEN='1//09nhOL0gpiD83CgYIARAAGAkSNwF-L9IrfVltzz8nXey8j3x_ZWo6aNcHXa4oQ5S3XAMX28F7qJaCdHo4lV6apuR8pDSf6CCEcpE'

discogs_client = DiscogsClient.Client('ExampleApplication/0.1', user_token=DISCOGS_USER_TOKEN)

In [60]:
def df_to_nested_dict(df):
    dict_result = (
        df.groupby('band')
        .apply(lambda x: x.groupby('album')['song'].apply(list).to_dict())
        .to_dict()
    )
    return dict_result

def find_band_exact(band_name):
    """Returns the artist object for searches that have only one exact match, ignoring case. If multiple results are returned, returns None"""
    band_results = discogs_client.search(band_name, type='artist')
    band_names = [result.name.lower() for result in band_results]
    band_name = band_name.lower()
    pattern = re.compile(fr'{band_name} \(\d\)')
    has_multiple = sum(map(lambda x: 1 if re.findall(pattern, x) else 0, band_names)) > 1

    if band_names.count(band_name) == 1 and not has_multiple:
        return band_results[band_names.index(band_name)]
    return None

def find_band_by_albums(band_name, albums):
    """Returns the artist object for searches with multiple results, by checking the album list. Returns the artist if any albums match the given list, otherwise None"""
    band_results = discogs_client.search(band_name, type='artist')
    albums = [album.lower() for album in albums]

    for band in band_results:
        releases = [release.title.lower() for release in band.releases]
        if any(set(releases).intersection(set(albums))):
            return band
    return None



<Artist 10670 'Covenant'>

In [103]:
df_raw = pd.read_pickle('with_discogs_ids.pkl')
df_multiple = df_raw.loc[df_raw['discogs_id'] == 'None']
data_dict = df_to_nested_dict(df_multiple)

# for band, albums in data_dict.items():
#     albums = list(albums.keys())
#     found_band = find_band_by_albums(band, albums)
#     print(found_band, band, albums)



In [97]:
from thefuzz import fuzz
fuzz.token_set_ratio('Unreal/Real', 'Unreal / Real')


100

In [ ]:
band = 'Grendel'
albums = list(data_dict[band].keys())
found_band = find_band_by_albums(band, albums)
[release.title for release in found_band.releases]

In [116]:
albums_discogs = ['Inhumane Amusement',
 'End Of Ages',
 'Prescription : Medicide',
 'Soilbleed',
 'Harsh Generation',
 'Chemicals + Circuitry',
 'Timewave Zero',
 'Age Of The Disposable Body',
 'Brace The Storm',
 'Ascending The Abyss']
albums_db = ['Prescription: Medicide', 'V/A - Storsequenz - Compilation CD']

set(albums_discogs).intersection(set(albums_db))

from thefuzz import process
for album in albums_db:
    candidates = [pair for pair in process.extract(album, albums_discogs, limit=100) if pair[1] > 50]
    print(candidates)


[('Prescription : Medicide', 98)]
[]


In [119]:
data = {'Grendel': {'Prescription: Medicide': ['Kurtz', 'Construct Constriction', 'Dream'], 'V/A - Storsequenz - Compilation CD': []}}
for band, albums_dict in data.items():
    dg_band_results = discogs_client.search(band, type='artist')
    for dg_band in dg_band_results:
        print(band)
        for album, tracks in albums_dict.items():
            print(album)
            print(tracks)

Grendel
Prescription: Medicide
['Kurtz', 'Construct Constriction', 'Dream']
V/A - Storsequenz - Compilation CD
[]


In [117]:
data_dict['Grendel']['Prescription: Medicide']

['Kurtz', 'Construct Constriction', 'Dream']

In [114]:
band = 'Battery'
band_results = discogs_client.search(band, type='artist')
band_names = [band.name for band in band_results]
process.extract(band, band_results, limit=100)

[(<Artist 268201 'Battery (3)'>, 90),
 (<Artist 74425 'Battery'>, 90),
 (<Artist 4544702 'Battery (9)'>, 90),
 (<Artist 319391 'Love Battery'>, 90),
 (<Artist 39241 'DJ Battery Brain'>, 90),
 (<Artist 75346 'Id Battery'>, 90),
 (<Artist 174020 'Battery Cage'>, 90),
 (<Artist 613708 'Piero Battery'>, 90),
 (<Artist 1987109 'Battery (5)'>, 90),
 (<Artist 1935152 'Battery Crémil'>, 90),
 (<Artist 113714 'Battery Park'>, 90),
 (<Artist 3137143 'Battery (7)'>, 90),
 (<Artist 2303642 'Battery 03'>, 90),
 (<Artist 1522684 'Battery Boys'>, 90),
 (<Artist 75111 'Battery Operated'>, 90),
 (<Artist 4649176 'Simon Battery'>, 90),
 (<Artist 1852049 'Dead Battery Productions'>, 90),
 (<Artist 59739 'Assult N Battery'>, 90),
 (<Artist 3290627 'Battery (8)'>, 90),
 (<Artist 3790771 'Assault & Battery (3)'>, 90),
 (<Artist 127276 'Battery (2)'>, 90),
 (<Artist 1544674 'Christophe Battery'>, 90),
 (<Artist 281771 'Battery 9'>, 90),
 (<Artist 2475577 'Assault & Battery (2)'>, 90),
 (<Artist 427040 '666 V

In [31]:
json_result = json.dumps(df_to_nested_dict(df_bands))

# with open('band_album_song_data.json', 'w') as file:
#     file.write(json_result)

with open('band_album_song_data.json', 'r') as file:
    json_data = json.load(file)

In [35]:
# Maps the artist to artist ID

# df_bands['discogs_id'] = ''

# i = 0
# for band in df_bands['band'].drop_duplicates():
#     try:
#         band_obj = find_band(band)
#     except Exception:
#         continue
#     band_id = band_obj.id if band_obj else None
#     df_bands.loc[df_bands['band'] == band, 'discogs_id'] = str(band_id)
#     print(i, band, band_obj, band_id)
#     i+=1

# display(df_bands)
